# Pipeline for remaining time prediction

In [1]:
import xgboost as xgb
import os
import sys
import pickle
import pandas as pd
import numpy as np
from joblib import dump, load
import copy

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torch.nn import functional as F

from sklearn.metrics import mean_absolute_error
import importlib.util

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.Trainer import CaseDataSet
from src.Trainer import RemTimePredictor
from src.Model import DLModels

from sklearn import preprocessing
from sklearn.metrics import f1_score
    


torch_device = "cpu"
device_package = torch.cpu
if importlib.util.find_spec("torch.backends.mps") is not None:
    if torch.backends.mps.is_available():
        torch_device = torch.device("mps")
        device_package = torch.mps
if torch.cuda.is_available():
    torch_device = torch.device("cuda")
    device_package = torch.cuda
    
torch_device

## XGB Predictor

In [2]:
encoding = "Agg_Mean"
train = CaseDataSet.CaseDataset(project_data_path="../../data/", input_data="dummy", data_version="_train",
                                    feature_list=["Activity", "LapseTime"],
                                    encoding=encoding, label="RemTime")

val = CaseDataSet.CaseDataset(project_data_path="../../data/", input_data="dummy", data_version="_val",
                                    feature_list=["Activity", "LapseTime"],
                                    encoding=encoding, label="RemTime")

test = CaseDataSet.CaseDataset(project_data_path="../../data/", input_data="dummy", data_version="_test",
                                    feature_list=["Activity", "LapseTime"],
                                    encoding=encoding, label="RemTime")

In [6]:
t1 = RemTimePredictor.XGBTrainer(train, val, model_path="../../models/test_remtime")
t1.train()
print(t1.score())
t1.save_model()

In [7]:
t2 = RemTimePredictor.XGBTrainer(train, test, model_path="../../models/test_remtime")
t2.load_model()
print(t2.score())

## LSTM

In [5]:
encoding = "All"
train = CaseDataSet.CaseDataset(project_data_path="../../data/", input_data="dummy", data_version="_train",
                                    feature_list=["Activity", "LapseTime"],
                                    encoding=encoding, label="RemTime")

val = CaseDataSet.CaseDataset(project_data_path="../../data/", input_data="dummy", data_version="_val",
                                    feature_list=["Activity", "LapseTime"],
                                    encoding=encoding, label="RemTime")

test = CaseDataSet.CaseDataset(project_data_path="../../data/", input_data="dummy", data_version="_test",
                                    feature_list=["Activity", "LapseTime"],
                                    encoding=encoding, label="RemTime")

In [7]:
# Hyperparameters
input_size = 8  # The number of expected features in the input x
hidden_size = 256  # The number of features in the hidden state h
num_layers = 1  # Number of recurrent layers
num_classes = 1 
learning_rate = 0.001


model = DLModels.SimpleLSTM(input_size, hidden_size, num_layers, num_classes).to(torch_device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss = nn.L1Loss()

In [8]:
trained_model, train_score, test_score = RemTimePredictor.train_model(model, optimizer, loss, loss, train, val,
            100, torch_device, device_package, print_iter=True)

In [9]:
RemTimePredictor.evaluate_model(trained_model, test, torch_device, device_package, batch_size=100)